# Proyecto 1 - Inteligencia de negocios ISIS 3301

Por:

- Juan Sebastián Ramírez 201923800 

- Andrés Santiago Triana 201923265 

- Gabriela García 201912531 

## Comprensión del negocio y enfoque analítico



## Comprensión de los datos y preparación de los datos 

Para poder realizar un modelo predictivo de la información de los pacientes es muy importante enteder los datos dados por el negocio.

### Importación e Instalación de librerias

Debido a que en este proyecto es necesario realizar procesamiento de texto, es necesario instalar ciertas librerias para el manejo del lenguaje inglés. (Contractions, inflex, etc.)

In [1]:
# Librería para manejar las contracciones que se presentan en el inglés.
!pip install contractions

In [2]:
# librería para manejar las flexiones gramaticales en el idioma inglés.
!pip install inflect
!pip install pandas-profiling==2.7.1

In [3]:
pip install pandas==1.2.0

Note: you may need to restart the kernel to use updated packages.


In [4]:
# librería Natural Language Toolkit, usada para trabajar con textos 
import nltk
# Punkt permite separar un texto en frases.
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/santiagotriana/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
# Descarga todas las palabras vacias, es decir, aquellas que no aportan nada al significado del texto
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/santiagotriana/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
# Descarga de paquete WordNetLemmatizer, este es usado para encontrar el lema de cada palabra
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/santiagotriana/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True


Además es necesario importar las librerias necesarias para la realización del proyecto.

In [7]:
# Librerías para manejo de datos
import math
import pandas as pd
pd.set_option('display.max_columns', 25) # Número máximo de columnas a mostrar
pd.set_option('display.max_rows', 50) # Numero máximo de filas a mostar
import numpy as np
np.random.seed(3301)
import pandas as pd
# Para preparar los datos
from sklearn.preprocessing import LabelEncoder, StandardScaler,MinMaxScaler
# Para crear el arbol de decisión 
from sklearn.tree import DecisionTreeClassifier 
# Para realizar la separación del conjunto de aprendizaje en entrenamiento y test.
from sklearn.model_selection import train_test_split
# Para evaluar el modelo
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import plot_confusion_matrix
# Para búsqueda de hiperparámetros
from sklearn.model_selection import GridSearchCV
# Para la validación cruzada
from sklearn.model_selection import KFold 
#Librerías para la visualización
import matplotlib.pyplot as plt
# Seaborn
import seaborn as sns 
from sklearn import tree
import seaborn as sns; sns.set()  # for plot styling

from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import silhouette_samples, silhouette_score
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D # for 3D plots

from sklearn.mixture import GaussianMixture
from sklearn.cluster import AgglomerativeClustering

from pandas_profiling import ProfileReport

import re, string, unicodedata
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer


from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix, plot_precision_recall_curve
from sklearn.base import BaseEstimator, ClassifierMixin

import matplotlib.pyplot as plt

Además establecemos las siguientes funciones que nos van a ayudar más adelante en cada uno de los pasos a seguir. 

In [8]:
def remove_non_ascii(words):
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    new_words = []
    for word in words:
        new_word = word.lower()
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    p = inflect.engine()
    new_words = []
    if(words==None): return new_words
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

stop_words = set(stopwords.words('english'))

def remove_stopwords(words):
    new_words =[]
    for word in words:
        if word not in stop_words:
            new_words.append(word)
    return new_words

def preprocessing(words):
    #print("1 - " + str(words) + "\n")
    words = to_lowercase(words)
    #print("2 - " + str(words) + "\n")
    words = replace_numbers(words)
    #print("3 - " + str(words) + "\n")
    words = remove_punctuation(words)
    #print("4 - " + str(words) + "\n")
    words = remove_non_ascii(words)
    #print("5 - " + str(words) + "\n")
    words = remove_stopwords(words)
    #print("6 - " + str(words) + "\n\n")
    return words


Ahora debemos importar los datos dados por el negocio

In [9]:
df_pacientes=pd.read_csv('datos/clinical_trials_on_cancer_data_clasificacion.csv', sep=',',
                         encoding = 'utf-8',
                         na_values={'-','NaN'})
df_pacientes.shape

(12000, 2)

In [10]:
df_pacientes.head(10)

,label,study_and_condition
0,__label__0,study interventions are Saracatinib . recurren...
1,__label__1,study interventions are Stem cell transplantat...
2,__label__0,study interventions are Lenograstim . recurren...
3,__label__0,study interventions are Doxorubicin . stage ii...
4,__label__1,study interventions are Poly I-C . prostate ca...
5,__label__1,study interventions are Bevacizumab . stage ii...
6,__label__0,study interventions are Carboplatin . stage ii...
7,__label__0,study interventions are Vidarabine . stage iv ...
8,__label__1,study interventions are Dexamethasone 21-phosp...
9,__label__1,study interventions are Microwave ablation . h...


In [11]:
data_t = df_pacientes
data_t = data_t.drop("study_and_condition",1)
data_t["study"] = [x.split('.')[0].strip() for x in df_pacientes["study_and_condition"]]
data_t["condition"] = [x.split('.')[1].strip() for x in df_pacientes["study_and_condition"]]
data_t["label"] = df_pacientes["label"]
data_t['label'].replace('__label__0','1',inplace=True)
data_t['label'].replace('__label__1','0',inplace=True)

data_t.head(10)


,label,study,condition
0,1,study interventions are Saracatinib,recurrent verrucous carcinoma of the larynx di...
1,0,study interventions are Stem cell transplantation,hodgkin lymphoma diagnosis and history of cong...
2,1,study interventions are Lenograstim,recurrent adult diffuse mixed cell lymphoma di...
3,1,study interventions are Doxorubicin,stage iii diffuse large cell lymphoma diagnosi...
4,0,study interventions are Poly I-C,prostate cancer diagnosis and unresolved iraes...
5,0,study interventions are Bevacizumab,stage iiic primary peritoneal cancer diagnosis...
6,1,study interventions are Carboplatin,stage iib uterine sarcoma diagnosis and patien...
7,1,study interventions are Vidarabine,stage iv childhood small noncleaved cell lymph...
8,0,study interventions are Dexamethasone 21-phosp...,non secretory multiple myeloma
9,0,study interventions are Microwave ablation,hepatic neoplasms diagnosis and severe renal d...


In [12]:
textos = data_t.copy()
textos['Conteo_study'] = [len(x) for x in textos['study']]
textos['Max_study'] = [[max([len(x) for x in i.split(' ')])][0] for i in textos['study']]
textos['Min_study'] = [[min([len(x) for x in i.split(' ')])][0] for i in textos['study']]

textos['Conteo_condition'] = [len(x) for x in textos['condition']]
textos['Max_condition'] = [[max([len(x) for x in i.split(' ')])][0] for i in textos['condition']]
textos['Min_condition'] = [[min([len(x) for x in i.split(' ')])][0] for i in textos['condition']]

# Se realiza un perfilamiento de los datos con la librería pandas profiling
ProfileReport(textos)

Summarize dataset: 23it [00:04,  5.24it/s, Completed]                           
Render HTML: 100%|████████████████████████████████| 1/1 [00:00<00:00,  1.62it/s]


In [13]:
textos.head(10)

,label,study,condition,Conteo_study,Max_study,Min_study,Conteo_condition,Max_condition,Min_condition
0,1,study interventions are Saracatinib,recurrent verrucous carcinoma of the larynx di...,35,13,3,212,15,2
1,0,study interventions are Stem cell transplantation,hodgkin lymphoma diagnosis and history of cong...,49,15,3,172,11,2
2,1,study interventions are Lenograstim,recurrent adult diffuse mixed cell lymphoma di...,35,13,3,191,12,2
3,1,study interventions are Doxorubicin,stage iii diffuse large cell lymphoma diagnosi...,35,13,3,230,9,2
4,0,study interventions are Poly I-C,prostate cancer diagnosis and unresolved iraes...,32,13,3,197,15,2
5,0,study interventions are Bevacizumab,stage iiic primary peritoneal cancer diagnosis...,35,13,3,175,12,2
6,1,study interventions are Carboplatin,stage iib uterine sarcoma diagnosis and patien...,35,13,3,107,11,2
7,1,study interventions are Vidarabine,stage iv childhood small noncleaved cell lymph...,34,13,3,90,10,2
8,0,study interventions are Dexamethasone 21-phosp...,non secretory multiple myeloma,50,13,3,30,9,3
9,0,study interventions are Microwave ablation,hepatic neoplasms diagnosis and severe renal d...,42,13,3,103,11,2


In [14]:
data_t.isnull().sum()

label        0
study        0
condition    0
dtype: int64

In [15]:
data_t.dtypes

label        object
study        object
condition    object
dtype: object

In [16]:
data_t['condition'] = data_t['condition'].apply(contractions.fix) #Aplica la corrección de las contracciones
data_t['study'] = data_t['study'].apply(contractions.fix) #Aplica la corrección de las contracciones

In [17]:
data_t['words'] = data_t['condition'].apply(word_tokenize).apply(preprocessing) #Aplica la eliminación del ruido
data_t.head(10)

,label,study,condition,words
0,1,study interventions are Saracatinib,recurrent verrucous carcinoma of the larynx di...,"[recurrent, verrucous, carcinoma, larynx, diag..."
1,0,study interventions are Stem cell transplantation,hodgkin lymphoma diagnosis and history of cong...,"[hodgkin, lymphoma, diagnosis, history, congen..."
2,1,study interventions are Lenograstim,recurrent adult diffuse mixed cell lymphoma di...,"[recurrent, adult, diffuse, mixed, cell, lymph..."
3,1,study interventions are Doxorubicin,stage iii diffuse large cell lymphoma diagnosi...,"[stage, iii, diffuse, large, cell, lymphoma, d..."
4,0,study interventions are Poly I-C,prostate cancer diagnosis and unresolved iraes...,"[prostate, cancer, diagnosis, unresolved, irae..."
5,0,study interventions are Bevacizumab,stage iiic primary peritoneal cancer diagnosis...,"[stage, iiic, primary, peritoneal, cancer, dia..."
6,1,study interventions are Carboplatin,stage iib uterine sarcoma diagnosis and patien...,"[stage, iib, uterine, sarcoma, diagnosis, pati..."
7,1,study interventions are Vidarabine,stage iv childhood small noncleaved cell lymph...,"[stage, iv, childhood, small, noncleaved, cell..."
8,0,study interventions are Dexamethasone 21-phosp...,non secretory multiple myeloma,"[non, secretory, multiple, myeloma]"
9,0,study interventions are Microwave ablation,hepatic neoplasms diagnosis and severe renal d...,"[hepatic, neoplasms, diagnosis, severe, renal,..."


In [21]:
def stem_words(words):
    ls = LancasterStemmer()
    rta = [ls.stem(word) for word in words]
    return rta

def lemmatize_verbs(words):
    wnl = WordNetLemmatizer()
    rta = [wnl.lemmatize(word) for word in words]
    return rta


def stem_and_lemmatize(words):
    stems = stem_words(words)
    lemmas = lemmatize_verbs(words)
    return stems + lemmas

data_t['words'] = data_t['words'].apply(stem_and_lemmatize) #Aplica lematización y Eliminación de Prefijos y Sufijos.
data_t.head()

,label,study,condition,words
0,1,study interventions are Saracatinib,recurrent verrucous carcinoma of the larynx di...,"[recur, verruc, carcinom, larynx, diagnos, pat..."
1,0,study interventions are Stem cell transplantation,hodgkin lymphoma diagnosis and history of cong...,"[hodgkin, lymphom, diagnos, hist, congenit, he..."
2,1,study interventions are Lenograstim,recurrent adult diffuse mixed cell lymphoma di...,"[recur, adult, diffus, mix, cel, lymphom, diag..."
3,1,study interventions are Doxorubicin,stage iii diffuse large cell lymphoma diagnosi...,"[stag, ii, diffus, larg, cel, lymphom, diagnos..."
4,0,study interventions are Poly I-C,prostate cancer diagnosis and unresolved iraes...,"[prost, cant, diagnos, unresolv, ira, follow, ..."
